# Fase 2 – Inventário de dados iniciais
Notebook para mapear todos os arquivos em `dados/dados_iniciais` como preparação do manifesto unificado.

## Objetivos imediatos
- definir `fase_1_diagnostico` como raiz lógica do trabalho;
- varrer recursivamente `dados/dados_iniciais/` reunindo metadados por arquivo;
- persistir o inventário em `dados/manifestos_inicial/inventario_dados_iniciais.csv`;
- exibir resumo com contagem e tamanho agregado por `pasta_raiz`.


In [1]:
from __future__ import annotations
from pathlib import Path
from dataclasses import dataclass
from datetime import datetime, timezone
from typing import Iterable, List, Dict
import hashlib
import os
import pandas as pd

In [2]:
def find_fase1_root(start: Path) -> Path:
    for candidate in (start, *start.parents):
        if candidate.name == 'fase_1_diagnostico':
            return candidate
    raise RuntimeError('Não foi possível localizar a pasta fase_1_diagnostico a partir deste notebook.')

CURRENT_DIR = Path.cwd().resolve()
FASE1_ROOT = find_fase1_root(CURRENT_DIR)
DADOS_INICIAIS_DIR = FASE1_ROOT / 'dados' / 'dados_iniciais'
OUTPUT_CSV = FASE1_ROOT / 'dados' / 'manifestos_inicial' / 'inventario_dados_iniciais.csv'

FASE1_ROOT, DADOS_INICIAIS_DIR.exists(), OUTPUT_CSV

(PosixPath('/home/wilson/Maringa/fase_1_diagnostico'),
 True,
 PosixPath('/home/wilson/Maringa/fase_1_diagnostico/dados/manifestos_inicial/inventario_dados_iniciais.csv'))

In [3]:
@dataclass
class FileRecord:
    path_local: str
    pasta_raiz: str
    nome_arquivo: str
    extensao: str
    tamanho_bytes: int
    data_modificacao: str
    hash_sha256: str

def compute_sha256(path: Path, chunk_size: int = 1 << 20) -> str:
    digest = hashlib.sha256()
    with path.open('rb') as stream:
        for chunk in iter(lambda: stream.read(chunk_size), b''):
            digest.update(chunk)
    return digest.hexdigest()

def iter_file_records(base_dir: Path) -> Iterable[FileRecord]:
    for file_path in base_dir.rglob('*'):
        if not file_path.is_file():
            continue
        rel_to_root = file_path.relative_to(FASE1_ROOT)
        rel_to_dados = file_path.relative_to(base_dir)
        pasta_raiz = rel_to_dados.parts[0] if rel_to_dados.parts else ''
        stat = file_path.stat()
        data_modificacao = datetime.fromtimestamp(stat.st_mtime, tz=timezone.utc).isoformat()
        hash_value = compute_sha256(file_path)
        yield FileRecord(
            path_local=str(rel_to_root).replace(os.sep, '/'),
            pasta_raiz=pasta_raiz,
            nome_arquivo=file_path.name,
            extensao=file_path.suffix.lower(),
            tamanho_bytes=stat.st_size,
            data_modificacao=data_modificacao,
            hash_sha256=hash_value,
        )

In [4]:
records = list(iter_file_records(DADOS_INICIAIS_DIR))
df = pd.DataFrame([record.__dict__ for record in records])
df = df.sort_values(['pasta_raiz', 'nome_arquivo']).reset_index(drop=True)
df.head()

,path_local,pasta_raiz,nome_arquivo,extensao,tamanho_bytes,data_modificacao,hash_sha256
0,dados/dados_iniciais/Consumo Fornos/2018_F2_Co...,Consumo Fornos,2018_F2_Consumo.csv,.csv,28950526,2025-04-29T15:22:50+00:00,7d942c3e034dca9818f3cda2f5a5ab173b370087e66acd...
1,dados/dados_iniciais/Consumo Fornos/2018_F3_Co...,Consumo Fornos,2018_F3_Consumo.csv,.csv,29621377,2025-04-29T15:23:02+00:00,9f93ce3cfbd1885b6aaea9ad42ac9df4c296996ae239a2...
2,dados/dados_iniciais/Consumo Fornos/2018_F4_Co...,Consumo Fornos,2018_F4_Consumo.csv,.csv,30486644,2025-04-29T15:23:16+00:00,8ee5c4e41d1f732906f78c17f624413a2412aba3f3ddcb...
3,dados/dados_iniciais/Consumo Fornos/2018_F5_Co...,Consumo Fornos,2018_F5_Consumo.csv,.csv,29401101,2025-04-29T15:23:30+00:00,1e60991d0d33868c7db1b5f06bb199f63475a3817a8ca7...
4,dados/dados_iniciais/Consumo Fornos/2019_F1_Co...,Consumo Fornos,2019_F1_Consumo.csv,.csv,42566405,2025-04-29T15:23:46+00:00,e2d6122f72312561518cd3dfbcfc425ca210fa1140a2e1...


In [5]:
OUTPUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(OUTPUT_CSV, index=False)
OUTPUT_CSV

PosixPath('/home/wilson/Maringa/fase_1_diagnostico/dados/manifestos_inicial/inventario_dados_iniciais.csv')

In [6]:
summary = (
    df.groupby('pasta_raiz')
    .agg(qtd_arquivos=('path_local', 'count'),
         tamanho_total_bytes=('tamanho_bytes', 'sum'))
    .sort_values('pasta_raiz')
)
summary

,qtd_arquivos,tamanho_total_bytes
pasta_raiz,,
Consumo Fornos,39,1132715472
Corridas,40,25655908
Dicionário,21,1345169
Eletrodo,1,20604
Informações Diária,40,3439738
Supervisorio Forno 4,2,914701033
Supervisorio Forno 5,2,48633162


In [7]:
{"total_arquivos": len(df), "tamanho_total_bytes": int(df['tamanho_bytes'].sum())}

{'total_arquivos': 145, 'tamanho_total_bytes': 2126511086}